In [1]:
# 
!git clone https://github.com/nqh-tq-32052503/BiLoRA.git
%cd /content/BiLoRA

Cloning into 'BiLoRA'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.  94% (101/107)
remote: Compressing objects: 100% (78/78), done.
remote: Total 107 (delta 47), reused 85 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 4.61 MiB | 10.85 MiB/s, done.
Resolving deltas: 100% (47/47), done.
/content/BiLoRA


In [6]:
!cd ..
!rm -rf /content/BiLoRA/

In [2]:
import json

with open("./configs/cifar100_bilora.json", "r") as f:
    args = json.load(f)

In [3]:
from utils import factory

/usr/local/lib/python3.12/dist-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.12/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [4]:
model = factory.get_model(args['model_name'], args)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Missing keys when loading pretrained weights: set()
Unexpected keys when loading pretrained weights: set()


In [1]:
import numpy as np
import torch

In [2]:
a = torch.nn.ModuleList([])

In [3]:
b = torch.nn.Embedding(12, 768)
a.append(b)

ModuleList(
  (0): Embedding(12, 768)
)

In [5]:
b.requires_grad_(True)

Embedding(12, 768)

In [8]:
labels = torch.randint(0, 10, (7, ))
cpe = torch.nn.Embedding(10, 768)
x = torch.rand(7, 196, 768)

In [16]:
labels

tensor([9, 3, 3, 3, 3, 8, 0])

In [ ]:
selected_cpe = torch.index_select(cpe.weight, 0, labels)
x = x + selected_cpe.unsqueeze(1)

In [17]:
selected_cpe.shape

torch.Size([7, 768])

In [19]:
a

tensor([[0.7313, 0.7603, 0.6274],
        [0.7329, 0.2975, 0.5860]])

In [20]:
b

tensor([[[0.7171, 0.2873, 0.8857],
         [0.6250, 0.8840, 0.2708]],

        [[0.7093, 0.3437, 0.5123],
         [0.4140, 0.3219, 0.5887]]])

In [18]:
a = torch.rand(2, 3)
b = torch.rand(2, 2, 3)
c = b + a.unsqueeze(1)
c

tensor([[[1.4483, 1.0476, 1.5131],
         [1.3562, 1.6444, 0.8981]],

        [[1.4422, 0.6413, 1.0984],
         [1.1469, 0.6194, 1.1747]]])

In [7]:
outputs[0].sum()

tensor(1.)

In [8]:
class_num=10

outputs_with_task = torch.zeros_like(outputs)[:,: class_num]
for idx, i in enumerate(targets//class_num):
    en, be =  class_num*i,  class_num*(i+1)
    outputs_with_task[idx] = outputs[idx, en:be]
predicts_with_task = outputs_with_task.argmax(dim=1)
predicts_with_task = predicts_with_task + (targets// class_num)* class_num

In [10]:
outputs[0]

tensor([0.0407, 0.0391, 0.0711, 0.0452, 0.0365, 0.0647, 0.0323, 0.0374, 0.0548,
        0.0645, 0.0359, 0.0667, 0.0547, 0.0469, 0.0372, 0.0323, 0.0844, 0.0340,
        0.0857, 0.0358])

In [9]:
predicts_with_task

tensor([18, 11, 11,  8, 11, 12,  4])

In [7]:
num_heads = 12
batch_size = 7
A = torch.randn(batch_size, num_heads, 197, 64)
B = torch.randn(num_heads, 50, 64)

In [14]:
X = torch.rand(7, 12, 197, 64)
Y = X[:, :, 0, :]
Z = X[:, :, 1:, :]

In [16]:
T = torch.cat((Y.unsqueeze(2), Z), dim=2)
T.shape

torch.Size([7, 12, 197, 64])

In [13]:
Y.shape

torch.Size([7, 12, 196, 64])

In [9]:
R = quantize(A, B)
R.shape

torch.Size([7, 12, 197, 64])

In [22]:
import torch

num_heads = 12
batch_size = 7
A = torch.randn(batch_size, num_heads, 197, 64)
B = torch.randn(num_heads, 50, 64)

dist_matrix = torch.cdist(A, B.unsqueeze(0), p=2) # Shape: (7, 12, 197, 50)
selected_indices = torch.argmin(dist_matrix, dim=-1)
B_expanded = B.unsqueeze(0).expand(batch_size, -1, -1, -1)
indices_expanded = selected_indices.unsqueeze(-1).expand(-1, -1, -1, 64)
R = torch.gather(B_expanded, 2, indices_expanded)

In [25]:
import torch.nn.functional as F

In [28]:
A = torch.rand(2, 12, 197, 197)
B = torch.rand(2, 12, 197, 197)

In [29]:
F.kl_div(A, B)

/usr/local/lib/python3.12/dist-packages/torch/nn/functional.py:3355: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


tensor(-0.4999)

In [ ]:

dist_matrix = torch.cdist(A, B, p=2) # Shape: (12, 197, 50)
selected_indices = torch.argmin(dist_matrix, dim=2)
indices_expanded = selected_indices.unsqueeze(-1).expand(-1, -1, 64)
R = torch.gather(B, 1, indices_expanded)

print(R.shape) # torch.Size([12, 197, 64])

torch.Size([12, 197, 64])


In [13]:
torch.index_select(B, 0, selected_indices).shape

torch.Size([197, 64])